In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Agent Observability - Plugins
 
In this notebook, you'll learn:
- How to develop the LoggingPlugin to add observability to an agent

## What is Agent Observability?

**🚨 The challenge:** Unlike traditional code, AI Agents are probabilistic—they might run perfectly five times and crash on the sixth because the LLM decided to format a date differently or "hallucinated" a parameter. Example:

```
User: "Book a flight to Paris for next Friday."
Agent (Internal Tool Call): book_flight(title="flight", date="next Friday")
System Response: ERROR: 400 Bad Request. Invalid format for 'date'. Expected YYYY-MM-DD, received 'next Friday'.
Agent (to User): "I'm sorry, I encountered a technical error and couldn't book the flight. Please try again later."
You: 😭 WHY?? Is it the prompt? Missing tools? API error?
```

**💡 The Solution:** Agent observability with ADK Logger Plugin gives you complete visibility into your agent's decision-making process. This plugin will hook into the agent's execution lifecycle phase to log exactly what prompts are sent to the LLM, which tools are available, how the model responds, and where failures occur.

```
DEBUG Log: LLM Request shows System Response: ERROR: 400 Bad Request. Invalid format for 'date'. Expected YYYY-MM-DD, received 'next Friday'.
You: 🎯 Aha! Wrong date format - easy fix!
```

## ADK Plugins

ADK Plugins are a mechanism for implementing logic that intercepts, modifies, and even controls the agent's execution lifecycle. The plugin is composed of callback hooks, which are specific methods in your Plugin class that you can implement to run code at a key moment. You have a choice between two modes of operation based on your hook's return value:

- To Observe: Implement a hook with no return value (None). This approach is for tasks such as logging or collecting metrics, as it allows the agent's workflow to proceed to the next step without interruption. For example, you could use after_tool_callback in a Plugin to log every tool's result for debugging.
- To Intervene: Implement a hook and return a value. This approach short-circuits the workflow. The Runner halts processing, skips any subsequent plugins and the original intended action, like a Model call, and use a Plugin callback's return value as the result. A common use case is implementing before_model_callback to return a cached LlmResponse, preventing a redundant and costly API call.
- To Amend: Implement a hook and modify the Context object. This approach allows you to modify the context data for the module to be executed without otherwise interrupting the execution of that module. For example, adding additional, standardized prompt text for Model object execution.

**This notebook covers:**

* ✅ Setting up logging configuration
* ✅ Create a broken agent.
* ✅ Understand how to implement logging in production

## Setup

### Install dependencies

To install and use ADK in your own Python development environment, you can do so by running:

```
pip install google-adk
```

**Note**: If you cloned the repo and you use poetry, you do not need to do any installation, Read the project README.md on steps to run this notebook within the poetry environment.

### Configure your Gemini API Key

This notebook uses the [Gemini API](https://ai.google.dev/gemini-api/), which requires an API key.

**1. Get your API key**

If you don't have one already, create an [API key in Google AI Studio](https://aistudio.google.com/app/api-keys).

**2. Add the key to your .env file**

Next, you will need to add your API key to your .env file as a new secret with the variable `GOOGLE_API_KEY=api_key`.

**3. Authenticate in the notebook**

Run the cell below to access the `GOOGLE_API_KEY` you just saved and set it as a variable for the notebook to use.

In [2]:
import os, time
from dotenv import load_dotenv
load_dotenv()  # Load variables from .env

GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')

### Set up logging and cleanup old files
Let's configure logging for our debugging session. The following cell makes sure we also capture other log levels, like DEBUG.

In [3]:
import logging

# Clean up any previous logs
for log_file in ["logger.log", "web.log", "tunnel.log", "agent_trace.log"]:
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"Cleaned up {log_file}")

# Configure logging with DEBUG log level.
logging.basicConfig(
    filename="agent_trace.log",
    level=logging.DEBUG,
    # 1. Add %(asctime)s to the start of the format string
    format="%(asctime)s %(filename)s:%(lineno)s %(levelname)s:%(message)s",
    # 2. (Optional) Define a clean date format
    datefmt="%Y-%m-%d %H:%M:%S"
)

print("✅ Logging configured")

Cleaned up agent_trace.log
✅ Logging configured


### Create the Travel Concierge Agent


**Goal:** Build a travel agent that provides personalized flight reservations for users.

The code intentionally creates an incorrect version of the agent to practice debugging!

### Agent definition

Next, let's create our travel agent. 
- We'll configure it as an `LlmAgent`, give it a name, model, and instruction.
- The `travel_agent` gets the user prompt and delegates the flight scheduling job to the `flight_agent`.
- Then, the agent uses the `book_flight_tool` tool to simulate the external API that fails if the date format is wrong.

In [4]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools.agent_tool import AgentTool

from google.genai import types
import random
import string

#A custom tool that inherits from AgentTool and handles the None case
class SafeAgentTool(AgentTool):
    async def run_async(self, args: dict, tool_context) -> str:
        try:
            # Call the original implementation
            result = await super().run_async(args=args, tool_context=tool_context)
            return result
        except TypeError as e:
            if "NoneType" in str(e):
                # Handle the specific case where the sub-agent returns no text
                return "Agent execution completed, but no text summary was returned."
            raise e

def generate_random_alphanumeric(length):
    characters = string.ascii_letters + string.digits
    return ''.join(random.choices(characters, k=length))

def book_flight_api(destination: str, date: str):
    """
    Simulates a legacy backend that is very strict about data formats.
    """
    import re
    # Strict validation: Date must be YYYY-MM-DD
    if not re.match(r"^\d{4}-\d{2}-\d{2}$", date):
        # This ValueError is what we want our LoggingPlugin to catch!
        raise ValueError(f"API Error: Invalid date format '{date}'. Backend expects strictly 'YYYY-MM-DD'.")
    
    length = 6
    reference_no = generate_random_alphanumeric(length)
    return f"Success: Flight to {destination} booked for {date}. PNR: {reference_no}"

# --- The "Broken" Tool Definition ---
def book_flight_tool(destination: str, date: str):
    """
    Books a flight ticket.
    
    Args:
      destination: The destination city (e.g., 'London').
      date: The intended date of departure. 
    
    Returns:
      Booking confirmation of the flight upon success, or error message on failure.
    """
    # INTENTIONAL BUG:
    # The docstring above is too vague. It says "date" but doesn't specify "YYYY-MM-DD".
    # The LLM will guess the format (often incorrectly as "Ded 12, 2025"), causing the API to crash.
    return book_flight_api(destination, date)

# --- Agent Configuration ---
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

# Flight agent
flight_agent = LlmAgent(
    name="flight_agent",
    model=Gemini(model="gemini-2.0-flash-lite", retry_options=retry_config),
    description="Handles flight bookings.",
    instruction="""
    You are a flight booking assistant. 
    Use the book_flight_tool to make reservations. 

    CRITICAL INSTRUCTIONS:
    1. If the tool returns a 'System Error', return the message to the root agent. Do NOT retry the tool!
    2. Do not ask the user for the date format, just interpret their natural language request.
    3. Return booking confirmation upon success.
    
    IMPORTANT: You MUST generate a final text summary for the user after the tool finishes. Do not return an empty response.
    """,
    tools=[book_flight_tool]
)


# Root agent
travel_agent = LlmAgent(
    name="travel_agent",
    model=Gemini(model="gemini-2.0-flash-lite", retry_options=retry_config),
    instruction="""
    You are the Lead Travel Coordinator.
    1. Delegate flight requests to the flight_agent.
    2. Pass the user's destination and date details exactly as provided. 
    3. Return both the booking confirmation and booking reference (PNP) if user asks.
    """,
    tools=[SafeAgentTool(agent=flight_agent)]
)
print("✅ Agent created")

✅ Agent created


### Create the Plugin class

We start by extending the BasePlugin class and add one or more callback methods, The callbacks are "atomic" (small, single-purpose) Python functions that act as hooks or interceptors. They pause the agent's flow at precise moments to run their code, then let the agent continue. Callbacks are grouped together to create a Plugin.

There are different kinds of callbacks that give you control over different "layers" of the application such as:

- **Agent Level (before/after_agent)**: Captures the start and end of the entire conversation turn.

- **Tool Level (before/after_tool)**: Captures when the agent decides to "do" something, like search Google or query a database. This is critical for debugging why an agent gave a wrong answer based on bad data.

- **Model Level (before/after_model)**: Captures the raw prompt sent to the LLM (e.g., GPT-4) and the raw response back. This can be used to track token usage (cost) and hallucination.

- **Error Level (on_model_error)**: Triggers specifically if the LLM crashes or times out, allowing you to log the error gracefully without crashing the whole app.

![image.png](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day4/plugins-callbacks.png)

*Image Source: [kaggle-genai-intensive-course/day-4](https://github.com/geminii01/kaggle-genai-intensive-course/tree/main/day-4), © 2025*

In [5]:
from typing import Any, Optional
from google.genai import types

from google.adk.agents.base_agent import BaseAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.events.event import Event
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse
from google.adk.tools.base_tool import BaseTool
from google.adk.tools.tool_context import ToolContext
from google.adk.plugins.base_plugin import BasePlugin
from google.adk.agents.invocation_context import InvocationContext

class LoggingPlugin(BasePlugin):
    """
    A custom plugin that tracks the invocation status by logging:
    - User messages and invocation context
    - Agent execution flow
    - LLM requests and responses
    - Tool calls with arguments and results
    - Events and final responses
    - Errors during model and tool execution
    
    Example:
      >>> logging_plugin = LoggingPlugin()
      >>> runner = Runner(
      ...     agents=[my_agent],
      ...     # ...
      ...     plugins=[logging_plugin],
      ... )
    """

    def __init__(self, name: str = "logging_plugin", logger: Optional[logging.Logger] = None):
        "Initialize the logging plugin."
        super().__init__(name)
        if logger:
            self.logger = logger
        else:
            self.logger = logging

    def _log(self, message: str, level: int = logging.INFO):
        "Helper to make logging cleaner."
        self.logger.log(level, message)

    def _format_content(self, content: Optional[types.Content]) -> str:
        "Format content for logging."""
        if not content or not content.parts:
          return "None"
        parts = []
        for part in content.parts:
            if part.text:
                text = part.text.strip()
                parts.append(f"text: '{text}'")
            elif part.function_call:
                parts.append(f"function_call: {part.function_call.name}")
            elif part.function_response:
                parts.append(f"function_response: {part.function_response.name}")
            elif part.code_execution_result:
                parts.append("code_execution_result")
            else:
                parts.append("other_part")
        return " | ".join(parts)

    def _format_args(self, args: dict[str, Any]) -> str:
        "Format arguments dictionary for logging."
        if not args:
          return "{}"
        
        formatted = str(args)
        return formatted

    async def on_user_message_callback(
        self, *, invocation_context: InvocationContext, user_message: types.Content
    ) -> Optional[types.Content]:
        "Log user message and invocation start."
        self._log(f"🚀 USER MESSAGE RECEIVED", level=logging.INFO)
        self._log(f"Session ID: {invocation_context.session.id} | Invocation ID: {invocation_context.invocation_id}", level=logging.INFO)
        # DEBUG
        self._log(f"User Content: {self._format_content(user_message)}", level=logging.DEBUG)
        if invocation_context.branch:
            self._log(f"Branch: {invocation_context.branch}", level=logging.DEBUG)
        return None

    async def before_run_callback(self, *, invocation_context: InvocationContext) -> Optional[types.Content]:
        "Log invocation start."
        agent_name = getattr(invocation_context.agent, "name", "Unknown")
        self._log(f"🏃 INVOCATION STARTING | Session ID: {invocation_context.session.id} | Invocation ID: {invocation_context.invocation_id}", level=logging.INFO)
        self._log(f"Starting Agent: {agent_name}", level=logging.INFO)
        return None

    async def on_event_callback(
          self, *, invocation_context: InvocationContext, event: Event
        ) -> Optional[Event]:
        "Log events yielded from the runner."
        self._log(f"📢 EVENT YIELDED (ID: {event.id})", level=logging.INFO)
        self._log(f"Author: {event.author} | Final: {event.is_final_response()}", level=logging.INFO)
        self._log(f"Content: {self._format_content(event.content)}", level=logging.DEBUG)
        if event.get_function_calls():
            func_calls = [fc.name for fc in event.get_function_calls()]
            self._log(f"Function Calls: {func_calls}", level=logging.INFO)
        if event.get_function_responses():
            func_responses = [fr.name for fr in event.get_function_responses()]
            self._log(f"Function Responses: {func_responses}", level=logging.INFO)
            self._log(f"Full Function Response Data: {event.get_function_responses()}", level=logging.DEBUG)
        if event.long_running_tool_ids:
            self._log(f"Long Running Tools: {list(event.long_running_tool_ids)}", level=logging.WARNING)
        return None

    async def after_run_callback(
          self, *, invocation_context: InvocationContext
        ) -> Optional[None]:
        "Log invocation completion."
        agent_name = getattr(invocation_context.agent, "name", "Unknown")
        self._log(
            f"✅ INVOCATION COMPLETED | ID: {invocation_context.invocation_id} | Final Agent: {agent_name}",
            level=logging.INFO
        )
        return None

    async def before_agent_callback(
          self, *, agent: BaseAgent, callback_context: CallbackContext
        ) -> Optional[types.Content]:
        "Log agent execution start."
        log_payload = {
            "agent_name": callback_context.agent_name, 
            "invocation_id": callback_context.invocation_id
        }
        
        if callback_context._invocation_context.branch:
            log_payload["branch"] = callback_context._invocation_context.branch
            
        self._log(f"🤖 AGENT STARTING: {log_payload}", level=logging.INFO)
        return None

    async def after_agent_callback(
          self, *, agent: BaseAgent, callback_context: CallbackContext
        ) -> Optional[types.Content]:
        "Log agent execution completion."
        log_payload = {
            "agent_name": callback_context.agent_name, 
            "invocation_id": callback_context.invocation_id
        }
        self._log(f"🤖 AGENT COMPLETED: {log_payload}", level=logging.INFO)
        return None

    async def before_model_callback(
        self, *, callback_context: CallbackContext, llm_request: LlmRequest
        ) -> Optional[LlmResponse]:
        "Log LLM request before sending to model."
        # extract System Instruction
        sys_instruction = "None"
        if llm_request.config and llm_request.config.system_instruction:
            sys_instruction = llm_request.config.system_instruction
        # extract Tool Names 
        tool_names = list(llm_request.tools_dict.keys()) if llm_request.tools_dict else ["None"]
        if tool_names == ["None"]:
            self._log(
                f"⚠️ Configuration Warning: Agent '{callback_context.agent_name}' has no tools enabled.", 
                level=logging.WARNING
            )
        # 3. Construct structured log message
        log_message = (
            f"🧠 LLM REQUEST DETAILS:\n"
            f"   • Agent: {callback_context.agent_name}\n"
            f"   • Model: {llm_request.model or 'default'}\n"
            f"   • Tools: {', '.join(tool_names)}\n"
            f"   • Sys Prompt: '{sys_instruction[:200]}'"
        )
        self._log(log_message, level=logging.DEBUG)
        return None

    async def after_model_callback(
        self, *, callback_context: CallbackContext, llm_response: LlmResponse
    ) -> Optional[LlmResponse]:
        "Log LLM response after receiving from model."
        agent_name = callback_context.agent_name
        # If the LLM failed, log as ERROR and exit immediately
        if llm_response.error_code and llm_response.error_code >= 400:
            self._log(
                f"❌ LLM ERROR for Agent '{agent_name}': "
                f"Code {llm_response.error_code}, Message: {llm_response.error_message}",
                level=logging.ERROR
            )
            return None
        # KPI & Usage Logging
        usage = llm_response.usage_metadata
        in_tokens = usage.prompt_token_count if usage else 0
        out_tokens = usage.candidates_token_count if usage else 0
        # Log event
        self._log(
            f"🧠 LLM RESPONSE received for Agent '{agent_name}'. "
            f"Usage: [In: {in_tokens}, Out: {out_tokens}]", 
            level=logging.INFO
        )
        # Verbose Debugging Data
        formatted_content = self._format_content(llm_response.content)
        debug_details = [
            f"📄 Full Content: {formatted_content}",
        ]
        # Add optional debug details only if they exist
        if llm_response.partial:
            debug_details.append(f"Partial Response: {llm_response.partial}")
        if llm_response.turn_complete:
            debug_details.append(f"Turn Complete: {llm_response.turn_complete}")
        # Log all debug info
        for detail in debug_details:
            self._log(detail, level=logging.DEBUG)
        return None

    async def before_tool_callback(
        self, *, tool: BaseTool, tool_args: dict[str, Any], tool_context: ToolContext,
    ) -> Optional[dict]:       
        "Log BEFORE the tool is executed"
        # INFO log
        log_header = (
            f"🔧 TOOL STARTING: [{tool.name}]\n"
            f"   • Agent: {tool_context.agent_name}\n"
            f"   • Call ID:    {tool_context.function_call_id}"
        )
        self._log(log_header, level=logging.INFO)
        # 2. DEBUG
        self._log(
            f"   • Arguments: {self._format_args(tool_args)}", level=logging.DEBUG
        )
        # 3. WARNING
        if not tool_args:
            self._log(
                f"⚠️  Warning: Tool '{tool.name}' was called without arguments.", 
                level=logging.WARNING
            )
        return None

    async def after_tool_callback(
        self, *, tool: BaseTool, tool_args: dict[str, Any], tool_context: ToolContext, 
        result: dict,
    ) -> Optional[dict]:
        """Log tool execution completion."""
        # INFO
        log_header = (
            f"🔧 TOOL COMPLETED: {tool.name}"
            f"   • Agent: {tool_context.agent_name}\n"
            f"   • Call ID:    {tool_context.function_call_id}"
        )
        self._log(log_header, level=logging.INFO)
        # DEBUG
        self._log(
            f"   • Result: {self._format_args(result)}", level=logging.DEBUG
        )
        return None

    async def on_model_error_callback(
        self, *, callback_context: CallbackContext, llm_request: LlmRequest, 
        error: Exception,
    ) -> Optional[LlmResponse]:
        "Log LLM error."
        error_message = (
            f"🧠 LLM ERROR in agent {callback_context.agent_name}"
            f"   • Exception details: {error}"
        )
        self._log(error_message, level=logging.ERROR)
        self._log(f"Payload causing error: {llm_request}", level=logging.DEBUG)
        fallback_content = types.Content(
            role="model",
            parts=[types.Part(text="Sorry, I'm currently having trouble connecting to my brain. Please try again later.")]
        )
        return LlmResponse(content=fallback_content)

    async def on_tool_error_callback(
        self, *, tool: BaseTool, tool_args: dict[str, Any], tool_context: ToolContext, 
        error: Exception,
    ) -> Optional[dict]:
        """Log tool error."""
        self._log(
            f"🔧 TOOL ERROR: '{tool.name}' failed during execution.\n"
            f"Context: Agent '{tool_context.agent_name}' (ID: {tool_context.function_call_id})\n"
            f"Error Message: {error}", level=logging.ERROR
        )
        self._log(
            f"🔧 TOOL ERROR DEBUG INFO - Arguments for '{tool.name}':\n"
            f"{self._format_args(tool_args)}", level=logging.DEBUG
        )
        return {"error": f"System Error: The tool failed to execute. Details: {str(error)}"}

### 📝 Key Insights: The `LoggingPlugin` Architecture

The `LoggingPlugin` class demonstrated above serves as a comprehensive **observability layer** for the Google ADK agent. Instead of scattering `print()` statements throughout the business logic, this plugin centralizes monitoring by leveraging ADK's **Lifecycle Callbacks**.

Here is a breakdown of how it enhances the agent's robustness:

#### 1. Full Observability (The "Black Box" Problem)

AI Agents are often "black boxes", inputs go in, and answers come out, but the intermediate reasoning is lost. This plugin utilizes the `BasePlugin` inheritance to hook into every critical stage:

* **Lifecycle Hooks:** It tracks the macro-flow (`before_run`, `after_run`) to measure total execution time per session.
* **Decision Hooks:** It tracks the micro-flow (`before_model`, `after_model`) to see exactly what the LLM "thought" before acting.
* **Action Hooks:** It tracks tool usage (`before_tool`, `after_tool`) to verify that the agent is passing the correct arguments to your APIs.

#### 2. Tiered Logging Strategy

The code implements a disciplined logging hierarchy that separates "operational noise" from "critical signals":

* **`INFO`**: Used for high-level flow (e.g., *“Agent Started,”* *“Tool Called”*). This is safe for production logs.
* **`DEBUG`**: Used for heavy data payloads (e.g., *Full JSON arguments, raw prompt text*). This is essential for development, but should be disabled in production to save storage.
* **`WARNING`**: Flags potential configuration issues, such as an agent defined with no tools (`before_model_callback`).

#### 3. Cost & Performance Tracking

In the `after_model_callback`, the plugin explicitly extracts `usage_metadata`:

```python
in_tokens = usage.prompt_token_count
out_tokens = usage.candidates_token_count

```

This is critical for **FinOps**. It allows you to calculate the exact cost of every interaction and identify expensive queries or inefficient prompts.

#### 4. Defensive Error Handling

The plugin acts as a safety net. In the `on_model_error_callback`, rather than letting the application crash if the LLM is unreachable, it intercepts the exception and returns a fallback message:

> *"Sorry, I'm currently having trouble connecting to my brain..."*

This ensures the user experience remains smooth even during backend failures.

#### 5. Context Propagation

Notice how almost every log includes `invocation_context.session.id`. In a production environment with thousands of concurrent users, this ID allows you to filter logs and reconstruct a single conversation thread across distributed systems.

### 💡 Pro Tip for Implementation

When adding this to your `Runner`, you can pass a custom logger instance. This allows you to route these specific agent logs to a separate file or a cloud monitoring service (like Google Cloud Logging) without interfering with your standard application logs.

```python
# Example Usage in Notebook
my_logger = logging.getLogger("agent_debugger")
plugin = LoggingPlugin(logger=my_logger)

```

**Note**: Plugin callback functions have precedence over callbacks implemented at the object level. This behavior means that Any Plugin callbacks code is executed before any Agent, Model, or tool object callbacks are executed. Furthermore, if a Plugin-level agent callback returns any value, and not an empty (None) response, the Agent, Model, or Tool-level callback is not executed (skipped). Read more about Plugin hooks [here](https://google.github.io/adk-docs/plugins/#plugin-callback-hooks).

### 3.4: Add LoggingPlugin to Runner

The following code creates the `InMemoryRunner`. This is used to programmatically invoke the agent.

**To use `LoggingPlugin` in the above travel agent,**
- Add it when initializing the `InMemoryRunner`.


In [6]:
from google.adk.runners import InMemoryRunner
from google.genai import types
import asyncio


# Instead of using the "root" (default) logger, this creates a specific channel named "agent_debugger" for the agent
# This allows you to filter these logs separately from system logs later if needed
debug_logger = logging.getLogger("agent_debugger")

runner = InMemoryRunner(
    agent=travel_agent,
    plugins=[
        LoggingPlugin(name="logging_plugin", logger=debug_logger)
    ],  # <---- 2. Add the plugin. Handles standard Observability logging across ALL agents
)

print("✅ Runner configured with LoggingPlugin")

✅ Runner configured with LoggingPlugin


### Run the agent

Let's now run our agent with the command in the next cell.

In [7]:
# EXECUTION LOOP
# Since ADK runners are often async, we need an async loop to interact with it.
async def run_interactive_session():
    print("✈️ Travel Agent is ready! (Type 'quit' to exit)")
    
    # Create a session ID (simple string for in-memory)
    session_id = "test-session-001"

    while True:
        # Get input from the user
        user_input = input("You: ")
        if user_input.lower() in ["quit", "exit"]:
            print("Exiting...")
            break

        # Send the message to the runner
        # The runner handles: Plugin Hooks -> Agent Thinking -> Tool Calling -> Response
        responses = await runner.run_debug(
            user_messages=user_input,
            session_id=session_id
        )

        # Print the final response from the agent
        for response in responses:
            # check for the parts
            if response.content and response.content.parts:
                for part in response.content.parts:
                    # Case 1: Standard Text
                    if part.text:
                        print(f"Agent: {part.text}")
                    
                    # Case 2: Tool Call (Function Execution)
                    # This happens if the agent decides to use a plugin/tool
                    elif part.function_call:
                        print(f"🛠️  Agent is calling tool: {part.function_call.name}")

In [8]:
# Run the async loop
# If you are in a standard .py script, use 'asyncio.run(run_interactive_session())'
# Book a flight to paris for next week friday, Why, did i miss any information?
await run_interactive_session()

✈️ Travel Agent is ready! (Type 'quit' to exit)


You:  Book a flight to paris for next week friday.



 ### Created new session: test-session-001

User > Book a flight to paris for next week friday.
travel_agent > Sorry, I'm currently having trouble connecting to my brain. Please try again later.
Agent: Sorry, I'm currently having trouble connecting to my brain. Please try again later.


You:  Why, did i miss any information?



 ### Continue session: test-session-001

User > Why, did i miss any information?
travel_agent > Sorry, I'm currently having trouble connecting to my brain. Please try again later.
Agent: Sorry, I'm currently having trouble connecting to my brain. Please try again later.


You:  quit


Exiting...


### Log Evaluation

To present the logs, we develop a Log Parser function that takes the raw text file created by the LoggingPlugin (agent_trace.log), reads it, and transforms it into a clean, readable table (DataFrame) so that we can analyze the agent's behavior without having to read thousands of lines of text.

In [9]:
import pandas as pd
import re

# Check the DEBUG logs from the broken agent
def parse_logs_to_df(log_file):
    log_pattern = re.compile(r'(?P<timestamp>[\d\- :]+) (?P<source>[\w\.]+:\d+) (?P<level>\w+):(?P<message>.*)')
    data = []

    try:
        with open(log_file, 'r') as f:
            for line in f:
                match = log_pattern.match(line)
                if match:
                    data.append(match.groupdict())
        
        df = pd.DataFrame(data)
        return df
    except FileNotFoundError:
        return pd.DataFrame()

In [10]:
# Load logs
df_logs = parse_logs_to_df("agent_trace.log")

if not df_logs.empty:
    # Set display options to read long messages
    pd.set_option('display.max_colwidth', 150)
    
    # 1. Filter for just the 'INFO' or 'ERROR' rows to show in your report
    display_df = df_logs[df_logs['level'].isin(['INFO', 'ERROR', 'WARNING'])]
    
    # Display the last 10 relevant events
    display(display_df.tail(10))
else:
    print("No logs found yet.")

,timestamp,source,level,message
157,2026-01-17 16:24:38,before_sleep.py:65,INFO,Retrying google.genai._api_client.BaseApiClient._async_request_once in 49.2584846706838 seconds as it raised ClientError: 429 RESOURCE_EXHAUSTED. ...
170,2026-01-17 16:25:28,_client.py:1740,INFO,"HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-lite:generateContent ""HTTP/1.1 429 Too Many Requests"""
175,2026-01-17 16:25:28,before_sleep.py:65,INFO,Retrying google.genai._api_client.BaseApiClient._async_request_once in 60.0 seconds as it raised ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'...
188,2026-01-17 16:26:29,_client.py:1740,INFO,"HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-lite:generateContent ""HTTP/1.1 429 Too Many Requests"""
193,2026-01-17 16:26:29,3475425242.py:43,ERROR,\U0001f9e0 LLM ERROR in agent travel_agent • Exception details:
196,2026-01-17 16:26:29,3475425242.py:43,INFO,"\U0001f9e0 LLM RESPONSE received for Agent 'travel_agent'. Usage: [In: 0, Out: 0]"
198,2026-01-17 16:26:29,3475425242.py:43,INFO,\U0001f4e2 EVENT YIELDED (ID: c438542a-1983-4ed4-be77-504bc42870b2)
199,2026-01-17 16:26:29,3475425242.py:43,INFO,Author: travel_agent | Final: True
201,2026-01-17 16:26:29,3475425242.py:43,INFO,"\U0001f916 AGENT COMPLETED: {'agent_name': 'travel_agent', 'invocation_id': 'e-7595405a-8bc9-40ca-9741-f9e97b79b81a'}"
202,2026-01-17 16:26:29,3475425242.py:43,INFO,\u2705 INVOCATION COMPLETED | ID: e-7595405a-8bc9-40ca-9741-f9e97b79b81a | Final Agent: travel_agent


In [11]:
display(display_df[display_df["level"]=="ERROR"])

,timestamp,source,level,message
91,2026-01-17 16:24:07,3475425242.py:43,ERROR,\U0001f9e0 LLM ERROR in agent travel_agent • Exception details:
193,2026-01-17 16:26:29,3475425242.py:43,ERROR,\U0001f9e0 LLM ERROR in agent travel_agent • Exception details:


**We've covered how to:**

- ✅ Debug agent failures through DEBUG logs 
- ✅ Scale observability with `LoggingPlugin` for production systems
- ✅ Understand when to use the different logging types

### Resources

**Refer to the ADK documentation to learn more about observability:**

- [ADK Observability Documentation](https://google.github.io/adk-docs/observability/logging/) - Complete guide to logging in ADK
- [Custom Plugin](https://google.github.io/adk-docs/plugins/) - Build your own Plugins
- [External Integrations](https://google.github.io/adk-docs/observability/cloud-trace/) - Explore external third-party observability integrations with ADK